In [2]:
%run ../utils.ipynb

[2024-01-12 16:56:46,406 INFO] [numexpr.utils.utils._init_num_threads] Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.


In [3]:
from yfpy import Data
from yfpy.logger import get_logger
from yfpy.query import YahooFantasySportsQuery

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import json

[2024-01-12 16:56:46,662 DEBUG] [matplotlib.__init__.wrapper] matplotlib data path: C:\Users\mikem\anaconda3\Lib\site-packages\matplotlib\mpl-data
[2024-01-12 16:56:46,666 DEBUG] [matplotlib.__init__.wrapper] CONFIGDIR=C:\Users\mikem\.matplotlib
[2024-01-12 16:56:46,668 DEBUG] [matplotlib.__init__.<module>] interactive is False
[2024-01-12 16:56:46,668 DEBUG] [matplotlib.__init__.<module>] platform is win32
[2024-01-12 16:56:46,710 DEBUG] [matplotlib.__init__.wrapper] CACHEDIR=C:\Users\mikem\.matplotlib
[2024-01-12 16:56:46,712 DEBUG] [matplotlib.font_manager.font_manager._load_fontmanager] Using fontManager instance from C:\Users\mikem\.matplotlib\fontlist-v330.json
[2024-01-12 16:56:46,712 DEBUG] [matplotlib.font_manager.font_manager._load_fontmanager] Using fontManager instance from C:\Users\mikem\.matplotlib\fontlist-v330.json


## Setup

In [4]:
auth_dir = project_dir+"\\auth\\"

In [5]:
Query_Auth_Ids

{'2015': [853925, 348],
 '2016': [810238, 359],
 '2017': [192551, 371],
 '2018': [799346, 380],
 '2019': [820083, 390],
 '2020': [394682, 399],
 '2021': [364325, 406],
 '2022': [564704, 414],
 '2023': [333146, 423]}

In [152]:
for year in Query_Auth_Ids:
    # Make a Yahoo query representing a single fantasy football season
    yahoo_query = YahooFantasySportsQuery(
        auth_dir,
        league_id=Query_Auth_Ids[year][0],
        game_code='nfl',
        game_id=Query_Auth_Ids[year][1]
    )
    
    # retrieve teams involved in that season and extract manager names (for reference) and IDs (for matchup method)
    weeks = [i for i in range(1, yahoo_query.get_league_metadata().end_week + 1)]
    # teams = yahoo_query.get_league_teams()
    # team_ids_list = []
    # for team in teams:
    #     manager = team.managers[0].nickname
    #     id = team.team_id
    #     info = {'manager_nickname': manager, 'id': id}
    #     team_ids_list.append(info)
    # team_ids_df = pd.DataFrame(team_ids_list)

    # create a dataframe for all weekly head to head matchups for the season
    try: 
        df = generate_weekly_matchup_data(yahoo_query,weeks)
        print(f"Generated matchup dataframe for {year}")
    except: 
        print('Error with the matchup generation, check rate limiting')
        break
    else:
        df.to_csv(f"{project_dir}\\data\matchup_summary\\{year}_matchups.csv", index=False)

Generating season matchup data for week 1
Generating season matchup data for week 2
Generating season matchup data for week 3
Generating season matchup data for week 4
Generating season matchup data for week 5
Generating season matchup data for week 6
Generating season matchup data for week 7
Generating season matchup data for week 8
Generating season matchup data for week 9
Generating season matchup data for week 10
Generating season matchup data for week 11
Generating season matchup data for week 12
Generating season matchup data for week 13
Generating season matchup data for week 14
Generating season matchup data for week 15
Generating season matchup data for week 16
Generated matchup dataframe for 2015
Generating season matchup data for week 1
Generating season matchup data for week 2
Generating season matchup data for week 3
Generating season matchup data for week 4
Generating season matchup data for week 5
Generating season matchup data for week 6
Generating season matchup data f

### Concatenate all matchup summary files and add season variable

In [6]:
files = [f"{project_dir}data\\matchup_summary\\raw_season\\{file}" for file in os.listdir(f"{project_dir}data\\matchup_summary\\raw_season\\") if ".csv" in file]

In [7]:
def add_season(file):
    sdf = pd.read_csv(file)
    sdf['season'] = file[-17:-13]
    return sdf

In [8]:
df = pd.concat( map(add_season, files), ignore_index=True)
    

### Tidy data and insert Win + UID

In [9]:
# remove extraneous characters from team names
df['team_name'] = df['team_name'].apply(lambda name: name[1:])
df['opponent_team_name'] = df['opponent_team_name'].apply(lambda name: name[1:])
df['team_name'] = df['team_name'].apply(lambda x: x.strip('"').strip("'"))
df['opponent_team_name'] = df['opponent_team_name'].apply(lambda x: x.strip('"').strip("'"))

# map team names to managers
df['manager_name'] = df['team_name'].apply(map_teams_to_name)
df['opponent_name'] = df['opponent_team_name'].apply(map_teams_to_name)

# rename confusing column names
df.rename({'season_id': 'in_season_manager_id'}, axis=1, inplace=True)
df.rename({'opponent_season_id': 'in_season_opponent_manager_id'}, axis=1, inplace=True)

# create a win column
df['win'] = np.where(df['points_for'] > df['points_against'], 1, 0)

# create a unique season.week ID that can be used as a UID
df['season_week'] = df['manager_name'] + '|' + df['season'].astype(str) + '|' + df['week'].astype(str)

In [169]:
df.head()

,week,team_name,in_season_manager_id,manager_name,opponent_team_name,in_season_opponent_manager_id,opponent_name,projected_points_for,points_for,projected_points_against,points_against,season,win,season_week
0,1,The Ice Cold Bruschi,1,Ricky,Jadeveon's Hit Men,3,Tim,105.32,123.80,94.97,100.87,2015,1,Ricky|2015|1
1,2,The Ice Cold Bruschi,1,Ricky,Thug Bungalow,5,Michael,94.91,106.43,96.95,102.10,2015,1,Ricky|2015|2
2,3,The Ice Cold Bruschi,1,Ricky,Thunder Crotches,7,Jake,98.54,101.23,102.01,89.87,2015,1,Ricky|2015|3
3,4,The Ice Cold Bruschi,1,Ricky,The Scientist,9,Kyle,102.49,113.67,107.05,84.53,2015,1,Ricky|2015|4
4,5,The Ice Cold Bruschi,1,Ricky,Immortal Duffmen,11,Pat,76.25,122.47,98.02,72.70,2015,1,Ricky|2015|5


In [10]:
df.to_csv(f"{project_dir}\\data\\matchup_summary\\to_date_matchup_summaries_tidy.csv", index=False)